# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [49]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [50]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("./output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,montbrison,45.6667,4.0833,12.19,85,99,1.00,FR,1683612655
1,1,carutapera,-1.1950,-46.0200,24.48,92,50,1.40,BR,1683612655
2,2,tiksi,71.6872,128.8694,-4.43,70,88,8.85,RU,1683612186
3,3,islamkot,24.6997,70.1770,38.36,16,0,5.25,PK,1683612656
4,4,puerto natales,-51.7236,-72.4875,3.43,84,13,2.05,CL,1683612181


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [51]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

filtr_ideal_weather = city_data_df[city_data_df['Max Temp'].between(21, 27)]
filtr_ideal_speed = filtr_ideal_weather[filtr_ideal_weather['Wind Speed']< 4.5]
filtr_ideal_cludiness = filtr_ideal_speed[filtr_ideal_speed['Cloudiness']== 0]

# Drop any rows with null values
no_null_filtered = pd.DataFrame.dropna(filtr_ideal_cludiness)

# Display sample data
no_null_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
154,154,galliano,29.4422,-90.2992,25.00,88,0,2.57,US,1683612184
166,166,cumberland,35.0502,-78.8664,23.28,83,0,4.12,US,1683612689
259,259,mu'minobod,38.1085,70.0285,22.35,37,0,1.52,TJ,1683612292
269,269,durant,33.9940,-96.3708,24.20,86,0,3.09,US,1683612588
274,274,crane,31.3974,-102.3501,24.68,41,0,3.80,US,1683612184
283,283,pelham,33.2857,-86.8100,22.45,95,0,1.54,US,1683612713
284,284,longboat key,27.4125,-82.6590,24.00,60,0,2.57,US,1683612413
300,300,spring hill,28.4769,-82.5255,22.77,91,0,0.00,US,1683612717
402,402,palmas,-10.2128,-48.3603,23.93,88,0,1.54,BR,1683612681
414,414,zhob,31.3411,69.4481,24.82,11,0,2.22,PK,1683612693


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = no_null_filtered[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
154,galliano,US,29.4422,-90.2992,88,
166,cumberland,US,35.0502,-78.8664,83,
259,mu'minobod,TJ,38.1085,70.0285,37,
269,durant,US,33.9940,-96.3708,86,
274,crane,US,31.3974,-102.3501,41,
283,pelham,US,33.2857,-86.8100,95,
284,longboat key,US,27.4125,-82.6590,60,
300,spring hill,US,28.4769,-82.5255,91,
402,palmas,BR,-10.2128,-48.3603,88,
414,zhob,PK,31.3411,69.4481,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [64]:
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]

In [65]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
galliano - nearest hotel: No hotel found
cumberland - nearest hotel: No hotel found
mu'minobod - nearest hotel: No hotel found
durant - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
pelham - nearest hotel: No hotel found
longboat key - nearest hotel: No hotel found
spring hill - nearest hotel: No hotel found
palmas - nearest hotel: No hotel found
zhob - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
timimoun - nearest hotel: No hotel found
ushtobe - nearest hotel: No hotel found
puerto penasco - nearest hotel: No hotel found
arys - nearest hotel: No hotel found
aswan - nearest hotel: No hotel found
moapa valley - nearest hotel: No hotel found
buka - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
154,galliano,US,29.4422,-90.2992,88,No hotel found
166,cumberland,US,35.0502,-78.8664,83,No hotel found
259,mu'minobod,TJ,38.1085,70.0285,37,No hotel found
269,durant,US,33.9940,-96.3708,86,No hotel found
274,crane,US,31.3974,-102.3501,41,No hotel found
283,pelham,US,33.2857,-86.8100,95,No hotel found
284,longboat key,US,27.4125,-82.6590,60,No hotel found
300,spring hill,US,28.4769,-82.5255,91,No hotel found
402,palmas,BR,-10.2128,-48.3603,88,No hotel found
414,zhob,PK,31.3411,69.4481,11,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols='Hotel Name'
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)